In [1]:
import requests
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np
import time

import seaborn
from matplotlib import pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn import utils
from scipy.stats import binom, poisson, norm

In [2]:
%load_ext sql

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:memory:

## Phase IV

### Introduction

Keikei & Yasmine

### Data description

get data description from Phase 2

### Preregistration statement

From Phase 3

### Data Analysis

Keikei & Yasmine

### Evaluation of significance

Keikei & Yasmine

### Interpretation and conclusions

Interpretation:
Hypothesis 1: Yasmine
Hypothesis 2: Yasmine
Hypothesis 3: Keikei
Conclusions: Yasmine

### Limitations

Keikei

### Acknowledgments

### Questions for reviewers

Yasmine & Keikei

#### Hypothesis #1

There's a correlation between an award in a given year being Latin and the unemployment rate fluctuating by a certain amount from 2009 to 2012.
- We will perform a pmf on the probability that given a song's award is Latin, the unemployment rate would fluctuate by a certain amount.
- We will also utilize a correlation matrix.

In [3]:
unemployaward_df = pd.read_csv('unemployaward_df.csv')

In [4]:
unemployaward_df['Latin'].value_counts

<bound method IndexOpsMixin.value_counts of 0      0
1      0
2      0
3      0
4      0
      ..
895    1
896    1
897    1
898    1
899    1
Name: Latin, Length: 900, dtype: int64>

#### Hypothesis #2

Given a host of variables in our unemployment data-- age range of those unemployed, their gender, and the cumulative unemployment rate-- we can predict whether or not Pitbull would make it onto the Billboard Charts.
 - We will do a train-test split logistical regression to see if we can train the data set on unemployment-- given a host of variables-- to see if we can predict Pitbull's chart success, as well as understand the relationship and effect different variables have on Pitbull chart success
 - Per feedback, we will utilize K-Folds instead

#### Hypothesis #3
We want to know the magnitude of the effect, if any, that the unemployment rate on a given month, both total average per month and by different categories, has on Pitbull popularity as measured by search trends. We do this by testing the hypothesis that the coefficients on the different unemployment measures are zero, having no effect on search term popularity. 
* We run a linear regression since the dependent variable, interest index, is discrete, ranging from 7 to 100

In [35]:
ucomplex_df=pd.read_csv('ucomplex_df.csv')
gtrends_df=pd.read_csv('gtrends_df.csv')

In [36]:
ucomplex_df.head(5)

,Unnamed: 0,date,unrate,u_men,u_women,u16_17,u18_19,u20_24,u25_34,u35_44,u45_54
0,0,1948-01-01,4.0,4.2,3.5,10.8,9.6,6.6,3.6,2.6,2.7
1,1,1948-02-01,4.7,4.7,4.8,15.0,9.5,8.0,4.0,3.2,3.4
2,2,1948-03-01,4.5,4.5,4.4,13.2,9.3,8.6,3.5,3.2,2.9
3,3,1948-04-01,4.0,4.0,4.1,9.9,8.1,6.8,3.5,3.1,2.9
4,4,1948-05-01,3.4,3.3,3.4,6.4,7.2,6.3,2.8,2.5,2.3


In [37]:
gtrends_df.head(5)

,Unnamed: 0,Interest Index,Date,Month,Year
0,1,7,2004-01-01,1,2004
1,2,9,2004-02-01,2,2004
2,3,10,2004-03-01,3,2004
3,4,13,2004-04-01,4,2004
4,5,20,2004-05-01,5,2004


In [38]:
%sql uandg_df << SELECT * FROM ucomplex_df INNER JOIN gtrends_df ON ucomplex_df.date=gtrends_df.Date
uandg_df.head(5)

Returning data to local variable uandg_df


,Unnamed: 0,date,unrate,u_men,u_women,u16_17,u18_19,u20_24,u25_34,u35_44,u45_54,Unnamed: 0,Interest Index,Date,Month,Year
0,672,2004-01-01,6.3,6.7,5.8,18.4,16.9,10.7,6.6,5.0,4.5,1,7,2004-01-01,1,2004
1,673,2004-02-01,6.0,6.4,5.5,17.9,16.6,10.1,6.2,4.9,4.3,2,9,2004-02-01,2,2004
2,674,2004-03-01,6.0,6.4,5.6,20.4,14.7,10.1,6.2,5.2,4.3,3,10,2004-03-01,3,2004
3,675,2004-04-01,5.4,5.6,5.1,21.1,13.3,8.7,5.4,4.3,4.0,4,13,2004-04-01,4,2004
4,676,2004-05-01,5.3,5.5,5.1,22.7,14.2,9.9,5.2,3.9,3.6,5,20,2004-05-01,5,2004


In [39]:
uandg_df.drop('Date',axis=1,inplace=True)

In [40]:
uandg_df.drop('Unnamed: 0',axis=1,inplace=True)

In [41]:
uandg_df.head(5)

,date,unrate,u_men,u_women,u16_17,u18_19,u20_24,u25_34,u35_44,u45_54,Interest Index,Month,Year
0,2004-01-01,6.3,6.7,5.8,18.4,16.9,10.7,6.6,5.0,4.5,7,1,2004
1,2004-02-01,6.0,6.4,5.5,17.9,16.6,10.1,6.2,4.9,4.3,9,2,2004
2,2004-03-01,6.0,6.4,5.6,20.4,14.7,10.1,6.2,5.2,4.3,10,3,2004
3,2004-04-01,5.4,5.6,5.1,21.1,13.3,8.7,5.4,4.3,4.0,13,4,2004
4,2004-05-01,5.3,5.5,5.1,22.7,14.2,9.9,5.2,3.9,3.6,20,5,2004


In [43]:
intcol=uandg_df.pop('Interest Index')
mcol=uandg_df.pop('Month')
ycol=uandg_df.pop('Year')
uandg_df.insert(0, 'Year', ycol)

In [45]:
uandg_df.insert(1, 'Month', mcol)
uandg_df.insert(2, 'Interest Index', intcol)

In [52]:
uandg_df.head(5)

,Year,Month,Interest Index,unrate,u_men,u_women,u16_17,u18_19,u20_24,u25_34,u35_44,u45_54
0,2004,1,7,6.3,6.7,5.8,18.4,16.9,10.7,6.6,5.0,4.5
1,2004,2,9,6.0,6.4,5.5,17.9,16.6,10.1,6.2,4.9,4.3
2,2004,3,10,6.0,6.4,5.6,20.4,14.7,10.1,6.2,5.2,4.3
3,2004,4,13,5.4,5.6,5.1,21.1,13.3,8.7,5.4,4.3,4.0
4,2004,5,20,5.3,5.5,5.1,22.7,14.2,9.9,5.2,3.9,3.6
